<a href="https://colab.research.google.com/github/QustaB/Prediction_of_Production_Sales/blob/main/Prediction_of_Production_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Production Sales

- Auther: Qusta Bahu

## Project Overview

## Load and Inspect Data

## Clean Data

## Exploratory Data Analysis

## Feature Inspection